# Basic Nighttime Lights Raster Processing


You can get these data packages via the ['sez' tag at the SDRDL Data Repository](https://data.sandiegodata.org/?s=sez)

In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rowgenerators as rg
import fiona
import rasterio
import rasterio.mask
from IPython.display import display 
from rasterio.plot import reshape_as_raster, reshape_as_image, show

%matplotlib inline
sns.set_context('notebook')


In [2]:
# Load the administrative boundaries
pkg = mp.open_package('http://library.metatab.org/berkeley.edu-administrative_boundaries-1.1.1.csv')

# Create Dataframes

admin = {}
for r in pkg.resources():
    # Only load second level admin boundary files
    if r.name.endswith('admin2'):
        p = r.name.split('_')
        # Store in dict with country name as key
        admin[p[0]] =  pkg.resource(r.name).geoframe()
        
for k,v in admin.items():
    print(k, v.shape)

skorea (229, 16)
china (344, 16)
vietnam (678, 16)
turkey (928, 16)
philippines (1652, 16)
indonesia (444, 16)
colombia (1065, 16)


In [3]:
# US Counties, for an additional comparison
counties=rg.geoframe('censusgeo://2019/05/US/county')

In [4]:
# Load two years of rasters
# See the rasterio documentation for more examples: 
# https://rasterio.readthedocs.io/en/latest/topics/masking-by-shapefile.html
def load_ntl(pkg, year, shapes=None):
    
    ref = pkg.reference(f'ntl{year}').resolved_url.get_resource().get_target()
    
    with rasterio.open(ref.fspath) as src:
        
        if shapes is not None:
            img, transform = rasterio.mask.mask(src, shapes, crop=True)
        else:
            img =  src.read()
            transform = None
            
        meta = src.meta
      
        
        return img, meta, transform

ntl = mp.open_package('http://library.metatab.org/figshare.com-harmonized_ntl-1.1.1.csv')
    
    
#ntl92, m92, t92 = load_ntl(ntl, 1992, tx.geometry)
#ntl18, m18, t18 = load_ntl(ntl, 2018, tx.geometry)
sk18, m18, t18 = load_ntl(ntl, 2018, admin['skorea'].geometry)

In [ ]:
rasters = [ load_ntl(ntl, 2018, [r.geometry])[0] for idx,r in admin['skorea'].iterrows()] 

In [ ]:
len(rasters)

In [ ]:
# Load a raster patch for each of the second-level administrative regions of
# South Korea and plot the first 100

#ntl = mp.open_package('http://library.metatab.org/figshare.com-harmonized_ntl-1.1.1.csv')

#rasters = [ load_ntl(ntl, 2018, [r.geometry])[0] for idx,r in admin['skorea'].iterrows()]   
    
fig, axes = plt.subplots(10,10, figsize=(10,10))
axes = np.ravel(axes)
for r, a in zip(rasters, axes):
    a.set_axis_off()
    show(r, ax=a)
    
fig.suptitle('2018 Nighttime Light Intensity\n100 Cities, Counties and District in South Korea', fontsize=14)

In [ ]:
sk98, m98, t98 = load_ntl(ntl, 1998, admin['skorea'].geometry)
sk18, m18, t18 = load_ntl(ntl, 2018, admin['skorea'].geometry)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,4))

ax[0].imshow(reshape_as_image(sk98))
ax[0].set_title('1998')
ax[1].imshow(reshape_as_image(sk18))
ax[1].set_title('2018')
ax[2].imshow(reshape_as_image(sk18-sk98))
ax[2].set_title('difference')
fig.suptitle("Nighttime Light Intensity of South Korea\n1998 and 2018", fontsize=18);
plt.tight_layout()

In [ ]:
show(sk18)